In [1]:
import OpenDartReader
import pandas as pd
import numpy as np
import time
import pickle

In [ ]:
api_key = '82f5064f12db8444b2bc1ab587e5cde026ad9fa0'
dart = OpenDartReader(api_key)

In [2]:
krxStockTickerDf = pd.read_csv('./datas/krx_stocks.csv', encoding='cp949')

In [3]:
stringExp2Ticker = "시장구분.str.contains('KOSPI')"
krxStockTickerDf.query(stringExp2Ticker, engine = 'python')

,종목코드,종목명,시장구분,소속부,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수
1,095570,AJ네트웍스,KOSPI,NaN,5220,-10,-0.19,5220,5250,5170,42089,219313330,244412379900,46822295
2,006840,AK홀딩스,KOSPI,NaN,18450,-200,-1.07,18250,18450,17850,38043,685399400,244417500450,13247561
6,027410,BGF,KOSPI,NaN,5010,-20,-0.40,5040,5040,4975,114268,570866205,479541122910,95716791
7,282330,BGF리테일,KOSPI,NaN,157000,0,0.00,156500,159500,155000,33568,5280593500,2713573242000,17283906
9,138930,BNK금융지주,KOSPI,NaN,8500,70,0.83,8400,8500,8340,1053087,8888505850,2770449591000,325935246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2599,069260,휴켐스,KOSPI,NaN,22750,0,0.00,22700,22850,22500,131458,2977784250,929987877000,40878588
2603,000540,흥국화재,KOSPI,NaN,3870,-15,-0.39,3885,3885,3815,24856,95583785,248619036150,64242645
2604,000547,흥국화재2우B,KOSPI,NaN,30000,-50,-0.17,28900,31000,28900,5520,166961150,4608000000,153600
2605,000545,흥국화재우,KOSPI,NaN,8190,30,0.37,8040,8300,8040,14054,115555070,6289920000,768000


In [ ]:
kospiTickerList = krxStockTickerDf.query(stringExp2Ticker, engine = 'python').종목코드.tolist()
#[Ticker, report_code, year]
waitingCounter = 0
jaemuDfList = []
for year in range(2018,2022):
    for ticker  in kospiTickerList:
        if year == 2021:
            for code in ['11013','11012','11014']:
                tempDf = dart.finstate_all(ticker, year, reprt_code = code)
                if tempDf is None :
                    pass
                else :
                    tempDf['corp_ticker'] = ticker
                    jaemuDfList.append(tempDf)
                    waitingCounter += 1
                    if waitingCounter == 900 :
                        time.sleep(30)
                        waitingCounter = 0
        elif year == 2020:
            for code in ['11011','11013','11012','11014']:
                tempDf = dart.finstate_all(ticker, year, reprt_code = code)
                if tempDf is None :
                    pass
                else :
                    tempDf['corp_ticker'] = ticker
                    jaemuDfList.append(tempDf)
                    waitingCounter += 1
                    if waitingCounter == 900 :
                        time.sleep(30)
                        waitingCounter = 0
        else : 
            tempDf = dart.finstate_all(ticker, year, reprt_code = '11011')
            if tempDf is None :
                pass
            else :
                tempDf['corp_ticker'] = ticker
                jaemuDfList.append(tempDf)
                waitingCounter += 1
                if waitingCounter == 900 :
                    time.sleep(30)
                    waitingCounter = 0


In [ ]:
with open('jaemuDfList.pickle', 'wb') as handle:
    pickle.dump(jaemuDfList, handle)

In [4]:
with open('jaemuDfList.pickle', 'rb') as handle:
    jaemuDfList = pickle.load(handle)

In [5]:
def reshapeEachCorpJaemu(df):

    df.loc[:,'account_nm'] = df.loc[:,'account_nm'].replace(' ','',regex=True)
    filter1 = "account_nm.str.contains('자산총계|자본총계|부채총계|매출액|매출원가|순이익|현금흐름$', regex = True)"
    filteredDf_1 = df.query(filter1, engine = 'python')
    
    filteredDf_2 = filteredDf_1[~filteredDf_1.sj_nm.isin(['자본변동표','현금흐름표','포괄손익계산서','손익계산서'])]
    
    filter3 = "sj_nm.str.match('현금흐름표') and (ord.str.len() == 1 and ord.str.match('0')) "                       # 현금흐름
    filteredDf_3 = filteredDf_1.query(filter3, engine = 'python')
    filteredDf_3 = filteredDf_3.assign(account_nm = '현금흐름') 
    
    filter4 = "sj_nm.str.contains('계산서') and (ord.str.len() == 1) and ord.str.match('0')"                         # 매출액
    filteredDf_4 = filteredDf_1.query(filter4, engine = 'python')
    filteredDf_4 = filteredDf_4.assign(account_nm = '매출액') 
    
    filter5 = "sj_nm.str.contains('계산서') and (ord.str.len() == 1) and ord.str.match('1')"                         # 매출원가
    filteredDf_5 = filteredDf_1.query(filter5, engine = 'python')
    filteredDf_5 = filteredDf_5.assign(account_nm = '매출원가') 

    filter6 = "sj_nm.str.contains('계산서') and (account_nm.str.len() <= 12) and account_nm.str.contains('순이익')"                                  # 당기순이익 
    filteredDf_6_temp = filteredDf_1.query(filter6, engine = 'python')                                                 
    filteredDf_6 = filteredDf_6_temp[~filteredDf_6_temp['account_nm'].str.contains('조정|외화|법인세|세전|주|중단|계속|지배|합계|귀속|지분')]
    filteredDf_6 = filteredDf_6.assign(account_nm = '순이익') 
    
    tempUnmeltedDf = pd.concat([filteredDf_2,filteredDf_3,filteredDf_4,filteredDf_5,filteredDf_6],sort=False)

    try : 

        amountMeltedDf = tempUnmeltedDf.melt(id_vars = ['rcept_no', 'reprt_code', 'bsns_year', 'corp_code', 'sj_div', 'sj_nm',\
                                                        'account_id', 'account_nm', 'account_detail','ord', 'corp_ticker']\
                                                            , value_vars = ['thstrm_amount','frmtrm_amount','bfefrmtrm_amount'])

        nameMeltedDf = tempUnmeltedDf.melt(value_vars = ['thstrm_nm','frmtrm_nm','bfefrmtrm_nm'])

        resultDf = pd.concat([amountMeltedDf,nameMeltedDf], axis = 1,sort=False)

    except:

        if 'frmtrm_amount' not in list(df.columns):

            amountMeltedDf = tempUnmeltedDf.melt(id_vars = ['rcept_no', 'reprt_code', 'bsns_year', 'corp_code', 'sj_div', 'sj_nm','account_id', 'account_nm', 'account_detail','ord', 'corp_ticker']\

            , value_vars = ['thstrm_amount'])

            nameMeltedDf = tempUnmeltedDf.melt(value_vars = ['thstrm_nm'])

            resultDf = pd.concat([amountMeltedDf,nameMeltedDf], axis = 1,sort=False)

        else :

            amountMeltedDf = tempUnmeltedDf.melt(id_vars = ['rcept_no', 'reprt_code', 'bsns_year', 'corp_code', 'sj_div', 'sj_nm','account_id', 'account_nm', 'account_detail','ord', 'corp_ticker']\

            , value_vars = ['thstrm_amount','frmtrm_amount'])

            nameMeltedDf = tempUnmeltedDf.melt(value_vars = ['thstrm_nm','frmtrm_nm'])

            resultDf = pd.concat([amountMeltedDf,nameMeltedDf], axis = 1,sort=False)


    return resultDf

In [6]:
accountNmFilteredDfLists = list(map(reshapeEachCorpJaemu, jaemuDfList))
accountNmFilteredDfconcated = pd.concat(accountNmFilteredDfLists)

In [7]:
accountNmFilteredDfconcated.columns = ['rcept_no', 'reprt_code', 'bsns_year', 'corp_code', 'sj_div', 'sj_nm','account_id', 'account_nm',\
                           'account_detail', 'ord', 'corp_ticker', 'variable_amount', 'amount', 'variable_account', 'kisu']

In [8]:
filter7 = "reprt_code.str.contains('11012|11013|11014') and variable_account.str.contains('frmtrm_nm|bfefrmtrm_nm')"                         # 각 분기보고서내 이전 기수 값 포함하는 Row 솎아내기
filteredDf_7 = accountNmFilteredDfconcated.query(filter7, engine = 'python')
outer_join = accountNmFilteredDfconcated.merge(filteredDf_7, how = 'outer', indicator = True)
antiJoinedaccountNmFilteredDfconcated = outer_join[~(outer_join._merge == 'both')].drop('_merge', axis = 1)

In [9]:
antiJoinedaccountNmFilteredDfconcated = antiJoinedaccountNmFilteredDfconcated[~antiJoinedaccountNmFilteredDfconcated['kisu'].str.contains('초')]
antiJoinedaccountNmFilteredDfconcated['kisu'] = antiJoinedaccountNmFilteredDfconcated['kisu'].str.replace('말','')

In [10]:
def recalculatedYear(x):
    if x['variable_account'] == 'frmtrm_nm':
        result = int(x['bsns_year']) - 1
    elif x['variable_account'] == 'bfefrmtrm_nm':
        result = int(x['bsns_year']) - 2
    else:
        result = int(x['bsns_year'])
    return result

In [11]:
antiJoinedaccountNmFilteredDfconcated['reCalculatedYear'] = antiJoinedaccountNmFilteredDfconcated.apply(recalculatedYear, axis = 1)
reCalculatedYeardJaemuResult = antiJoinedaccountNmFilteredDfconcated.loc[:,['reprt_code', 'reCalculatedYear', 'corp_code', 'account_nm', 'corp_ticker', 'amount', 'kisu']].drop_duplicates(ignore_index = True)
pivotFinalJaemuDf_v0 = reCalculatedYeardJaemuResult.pivot_table(index = ['corp_ticker','reCalculatedYear','kisu','reprt_code'],\
                            columns='account_nm',values = 'amount',aggfunc='first')

In [12]:
finalMergedJaemuAndPrice = pd.merge(pivotFinalJaemuDf_v0.reset_index(), krxStockTickerDf, left_on = 'corp_ticker', right_on = '종목코드')

In [13]:
finalMergedJaemuAndPrice.to_csv('./220209_result.csv', encoding = 'euc-kr')

In [ ]:
# commit test